<a href="https://colab.research.google.com/github/Mayukhmaity/DeepLearning_NLP/blob/main/Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## WORD EMBEDDING for Sentiment Analysis

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.6.0


In [3]:
## load the imdb review datasets
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete511RIG/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete511RIG/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
## segregate training and test set
train_data , test_data = data['train'], data['test']

#create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [6]:
##iterate over the train data to extract sentences and labels

for sent, label in train_data:
  train_sentences.append(str(sent.numpy().decode('utf8')))
  train_labels.append(label.numpy())

##iterate over the test set to extract sentences and labels

for sent, label in test_data:
  test_sentences.append(str(sent.numpy().decode('utf8')))
  test_labels.append(label.numpy())

In [7]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [8]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
embedding_size = 16
max_length = 150
trunc_type = 'post'
oov_tok = '<oov>'

In [10]:
tokenizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

## training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen= max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length)

In [11]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ''.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   11   26   75  571
    6  805 2354  313  106   19   12    7  629  686    6    4 2219    5
  181  584   64 1454  110 2263    3 3951   21    2    1    3  258   41
 4677    4  174  188   21  

# **Define the Neural Network with Embedding Layer**

1. Use the Sequencial API
2. Add an embedding input layer of input size equal to vocalbulary size
3. Add a flatten layer, and two dense layers

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_size, input_length= max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

## compile the model with loss function, optimizer and metrics

model.compile(loss='ninary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 14406     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 174,413
Trainable params: 174,413
Non-trainable params: 0
_________________________________________________________________


# **MODEL TRAINING**

In [20]:
num_epochs = 10

## train the model with training and validation set

model.fit(
    train_padded,
    train_labels,
    epochs=num_epochs,
    validation_data = (test_padded, test_labels)
)

Epoch 1/10


ValueError: ignored